# CALCULAR A MÉDIA MENSAL DOS DADOS DA NOAA

#### O script cálcula a média mensal dos dados NOAA de:
        
 - Temperatura da Superfície do Mar (sst - Sigla em Inglês)
 - Anômalia da TSM (anom)
 - Desvio padrão do erro estimado de TSM (err)
 - Concentração de gelo marinho (ice)

In [1]:
import os
import xarray as xr
import numpy as np

In [3]:
# Diretório onde estão os dados organizados como NOAA_SST/ano/mês/
input_dir = "NOAA_SST"
# Diretório para salvar os arquivos mensais
output_dir = "NOAA_SST_MENSAL"
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Loop por cada ano e mês disponíveis
for year in range(2000, 2026):
    for month in range(1, 13):
        # Limita até março de 2025
        if year == 2025 and month > 4:
            break

        # Caminho do diretório com os arquivos diários
        month_dir = os.path.join(input_dir, f"{year:04d}", f"{month:02d}")
        if not os.path.exists(month_dir):
            continue  # pula se o diretório não existe

        # Lista de arquivos .nc do mês
        arquivos_nc = sorted([f for f in os.listdir(month_dir) if f.endswith(".nc")])
        if not arquivos_nc:
            continue  # pula se não houver arquivos

        # Lista para armazenar os datasets diários
        datasets = []

        for nome_arquivo in arquivos_nc:
            caminho_arquivo = os.path.join(month_dir, nome_arquivo)
            try:
                ds = xr.open_dataset(caminho_arquivo)
                datasets.append(ds)
            except Exception as e:
                print(f"Erro ao abrir {caminho_arquivo}: {e}")

        # Concatena todos os arquivos do mês ao longo da dimensão 'time'
        try:
            ds_concat = xr.concat(datasets, dim='time')
        except Exception as e:
            print(f"Erro ao concatenar os arquivos de {year}-{month:02d}: {e}")
            continue

        # Calcula a média mensal para as variáveis desejadas
        variaveis_media = ['anom', 'err', 'ice', 'sst']
        media_mensal = ds_concat[variaveis_media].mean(dim='time', skipna=True)

        # Extrai variáveis constantes (lon, lat, zlev, time)
        constantes = {
            'lat': ds_concat['lat'],
            'lon': ds_concat['lon'],
            'zlev': ds_concat['zlev']
        }

        # Cria a coordenada de tempo única para o mês (meio do mês)
        tempo_mensal = np.datetime64(f"{year:04d}-{month:02d}-15")
        constantes['time'] = xr.DataArray([tempo_mensal], dims='time')

        # Adiciona a dimensão 'time' às médias
        for var in media_mensal.data_vars:
            media_mensal[var] = media_mensal[var].expand_dims('time')

        # Junta as variáveis médias com as constantes
        ds_mensal = xr.Dataset(
            data_vars=media_mensal.data_vars,
            coords={'time': constantes['time'],
                    'lat': constantes['lat'],
                    'lon': constantes['lon'],
                    'zlev': constantes['zlev']
                   }
        )

        # Nome do arquivo de saída
        nome_saida = f"NOAA_{year:04d}{month:02d}.nc"
        caminho_saida = os.path.join(output_dir, nome_saida)

        # Salva o novo arquivo .nc com compressão
        try:
            ds_mensal.to_netcdf(caminho_saida, format='NETCDF4', encoding={var: {'zlib': True} for var in ds_mensal.data_vars})
            print(f"✅ Média mensal salva: {caminho_saida}")
        except Exception as e:
            print(f"Erro ao salvar {caminho_saida}: {e}")

✅ Média mensal salva: NOAA_SST_MENSAL\NOAA_202001.nc
✅ Média mensal salva: NOAA_SST_MENSAL\NOAA_202504.nc
